In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression

In [16]:
hybrid_vars = [0,1, 2, 3, 4, 5, 6, 7, -3, -2, -1]

In [17]:
hybrid_model = TimeSeriesGradientBoosting(model1 = LinearRegression(), model2 = XGBRegressor(), model1_variables = hybrid_vars)

In [18]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

hybrid_xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', hybrid_model)
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [21]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithms=[hybrid_xgb_pipe, sgd_pipe], algorithm_name=["Gradient boosted decision tree", "Linear regression"], online=[False, False])
    return thesis


In [22]:
thesis = full_run()

Finished experiment! Elapsed time: 0.5059776306152344, total Elapsed time: 0.5059776306152344, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.5464060306549072, total Elapsed time: 1.0533816814422607, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.5224804878234863, total Elapsed time: 1.575862169265747, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.5257503986358643, total Elapsed time: 2.102609872817993, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished e

In [23]:
results = thesis.resultsToDF()

In [24]:
results[["Algorithm","SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Algorithm","Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                                      mean   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             5.358333   
                                                                Small             2.690000   
                                                 Half observed  Large            10.680833   
                                                                Small             3.324167   
                                                 Unobserved     Large             8.833333   
                                                                Small             3.047500   
                               No Drift          No Drift       No Drift          2.150000   
                               Sudden Drift      Fully observed Large             4.672500   
                                                                Small             2.635000   
                                                 Half observed  Large             4.032500   
                                                                Small             2.833333   
                                                 Unobserved     Large            12.063333   
                                                                Small             3.570000   
                               Sudden Shock      3 days         Large             2.890000   
                                                                Small             2.370000   
                                                 7 days         Large             2.757500   
                                                                Small             2.275000   
Linear regression              Incremental Drift Fully observed Large             3.055000   
                                                                Small             2.225833   
                                                 Half observed  Large            19.005000   
                                                                Small             5.440833   
                                                 Unobserved     Large            14.508333   
                                                                Small             4.692500   
                               No Drift          No Drift       No Drift          2.152500   
                               Sudden Drift      Fully observed Large             3.125833   
                                                                Small             2.275833   
                                                 Half observed  Large             4.147500   
                                                                Small             2.505833   
                                                 Unobserved     Large            21.241667   
                                                                Small             5.923333   
                               Sudden Shock      3 days         Large             2.292500   
                                                                Small             2.175000   
                                                 7 days         Large             2.307500   
                                                                Small             2.180000   

                                                                                            \
                                                                                       std   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             3.079078   
                                                                Small             0.567498   
                                                 Half observed  Large             8.076255   
                                          

In [10]:
results.sort_values("SMAPE", ascending=False)   

,Algorithm,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
298,Linear regression,6.0,Sudden Drift,Large,Important,Unobserved,23994.038881,127.58
175,Linear regression,NaN,Sudden Drift,Large,Important,Unobserved,13951.542613,114.69
257,Linear regression,3.0,Sudden Drift,Large,Important,Unobserved,13699.533142,113.60
315,Linear regression,6.0,Incremental Drift,Large,Important,Half observed,24929.315174,109.27
274,Linear regression,3.0,Incremental Drift,Large,Important,Half observed,14701.285495,99.51
...,...,...,...,...,...,...,...,...
47,Gradient boosted decision tree,0.0,Sudden Drift,Small,Unimportant,Fully observed,11.920706,2.13
88,Gradient boosted decision tree,3.0,Sudden Drift,Small,Unimportant,Fully observed,11.576972,2.13
36,Gradient boosted decision tree,NaN,No Drift,No Drift,No Drift,No Drift,11.239483,2.10
77,Gradient boosted decision tree,0.0,No Drift,No Drift,No Drift,No Drift,11.268753,2.09
